In [1]:
# DisulfideBond Playground
# Playing with the DisulfideBond class
# Author: Eric G. Suchanek, PhD.
# (c) 2022 Eric G. Suchanek, PhD., All Rights Reserved
# License: BSD
# Last Modification: 12/12/22
# Cα Cβ Sγ

%matplotlib inline

import matplotlib.pyplot as plt
import pandas as pd

from Bio.PDB import *

# location for PDB repository
PDB_ROOT = '/Users/egs/PDB/'

# location of cleaned PDB files - these are not stored in the repo
PDB_GOOD = '/Users/egs/PDB/good/'

# from within the repo 
PDB_REPO = '../pdb/'

# location of the compressed Disulfide .pkl files
MODELS = f'{PDB_ROOT}models/'

# for using from the repo we 
from proteusPy import *
from proteusPy.disulfide import *


ModuleNotFoundError: No module named 'proteusPy.disulfide'

In [4]:
# Comment these out since they take so long.
# DownloadDisulfides(pdb_home=PDB_ORIG, model_home=MODELS, reset=False)
# ExtractDisulfides(numb=-1, pdbdir=PDB, modeldir=MODELS, verbose=False, quiet=True)

PDB_SS = None
PDB_SS = DisulfideLoader(verbose=True, modeldir=MODELS)

ss_list = DisulfideList([], 'tmp')


Reading disulfides from: /Users/egs/PDB/models/PDB_all_ss.pkl
Disulfides Read: 8128
Reading disulfide dict from: /Users/egs/PDB/models/PDB_all_ss_dict.pkl
Reading Torsion DF /Users/egs/PDB/models/PDB_SS_torsions.csv.
Read torsions DF.
PDB IDs parsed: 1000
Total Space Used: 1761893 bytes.


In [5]:
ss1 = PDB_SS[0]
print(ss1)
ss2 = PDB_SS['4yys']
#print(ss2)

<Disulfide 4yys_22A_65A SourceID: 4yys Proximal: 22 A Distal: 65 A>


In [6]:
# load SS bonds by PDB ID
ss1 = PDB_SS['4yys']
print(ss1)
print(ss1[0].get_full_id())


[<Disulfide 4yys_22A_65A SourceID: 4yys Proximal: 22 A Distal: 65 A>, <Disulfide 4yys_56A_98A SourceID: 4yys Proximal: 56 A Distal: 98 A>, <Disulfide 4yys_156A_207A SourceID: 4yys Proximal: 156 A Distal: 207 A>, <Disulfide 4yys_22B_65B SourceID: 4yys Proximal: 22 B Distal: 65 B>, <Disulfide 4yys_56B_98B SourceID: 4yys Proximal: 56 B Distal: 98 B>, <Disulfide 4yys_156B_207B SourceID: 4yys Proximal: 156 B Distal: 207 B>]
(('4yys', 0, 'A', (' ', 22, ' ')), ('4yys', 0, 'A', (' ', 65, ' ')))


In [7]:
# you can loop over the IDList list and extract by ID
#
for id in PDB_SS.IDList[:5]:    # just show last 5
    # get the SS bonds for the given ID
    ssb = PDB_SS[id]
    numb_ss = len(ssb)
    print(f'ID: {id} has {numb_ss} Disulfides:')
    for bond in ssb:
        print(bond)
    print('\n')
    

ID: 4yys has 6 Disulfides:
<Disulfide 4yys_22A_65A SourceID: 4yys Proximal: 22 A Distal: 65 A>
<Disulfide 4yys_56A_98A SourceID: 4yys Proximal: 56 A Distal: 98 A>
<Disulfide 4yys_156A_207A SourceID: 4yys Proximal: 156 A Distal: 207 A>
<Disulfide 4yys_22B_65B SourceID: 4yys Proximal: 22 B Distal: 65 B>
<Disulfide 4yys_56B_98B SourceID: 4yys Proximal: 56 B Distal: 98 B>
<Disulfide 4yys_156B_207B SourceID: 4yys Proximal: 156 B Distal: 207 B>


ID: 1j5h has 2 Disulfides:
<Disulfide 1j5h_37A_47A SourceID: 1j5h Proximal: 37 A Distal: 47 A>
<Disulfide 1j5h_88A_93A SourceID: 1j5h Proximal: 88 A Distal: 93 A>


ID: 1mfe has 1 Disulfides:
<Disulfide 1mfe_137L_196L SourceID: 1mfe Proximal: 137 L Distal: 196 L>


ID: 1chv has 4 Disulfides:
<Disulfide 1chv_3S_21S SourceID: 1chv Proximal: 3 S Distal: 21 S>
<Disulfide 1chv_14S_38S SourceID: 1chv Proximal: 14 S Distal: 38 S>
<Disulfide 1chv_42S_53S SourceID: 1chv Proximal: 42 S Distal: 53 S>
<Disulfide 1chv_54S_59S SourceID: 1chv Proximal: 54 S Distal

In [8]:
ss_list = PDB_SS.getlist()
ss0 = ss_list[0]

print(ss0.proximal_residue_fullid)


('4yys', 0, 'A', (' ', 22, ' '))
